In [9]:

import random
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette('husl')
data=pd.read_csv("healthcare-dataset-stroke-data.csv")

def initialize_network(n_inputs, n_hidden, n_outputs):
    network=list()
    hidden_layer=[{'weights':[random.random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer=[{'weights':[random.random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

def activate(weights, inputs):
    activation=weights[-1]
    print(inputs[0])
    print(inputs[1])
    for i in range(len(weights)-1):
        print(weights[i])
        print(inputs[i])
        activation+=weights[i]*inputs[i]
    return activation

def transfer(activation):
    return 1/(1+math.exp(-activation))


def transfer_derivative(output):
    return output * (1-output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

            
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
 
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))


def shuffle(X):
    for i in (len(X)-1,0,-1):
        j=random.randint(0,i+1)
        temp=X.iloc[i].copy()
        X.iloc[i]=X.iloc[j].copy()
        X.iloc[j]=temp
            
    return X

def normalize(x):
    for i in x.select_dtypes(exclude="object").columns.tolist():
        c=x.loc[:, i]
        Max=max(c)
        Min=min(c)
        for j in range(len(x)):
            nor = (x.at[j,i]-Min)/(Max-Min)
            x.at[j,i]=nor
    return x
# Split a dataset into k folds
def split(x):
    split=int(len(x)*0.6)
    training=x[:split]
    validation=x[split:]
    return [training,validation]
    
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, *args):
    train_set, test_set = split(dataset)
    scores=[]
    for i in range(len(test_set)):
        predicted = algorithm(train_set, test_set, *args)
        actual = test_set.iloc[i][11]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train.iloc[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    print(n_inputs)
    print(n_outputs)
    print(n_hidden)
    print(train)
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)


normalize(data)
del data['id']

data['gender']=data['gender'].apply(lambda x : 1 if x=='Male' else 0) 
data["Residence_type"] = data["Residence_type"].apply(lambda x: 1 if x=="Urban" else 0)
data["ever_married"] = data["ever_married"].apply(lambda x: 1 if x=="Yes" else 0)
data["work_type"]=data["work_type"].apply(lambda x: 1 if x=="Private" else( 0.5 if x=="Self-employed" else 0))
data=data[data['smoking_status']!='Unknown']
data["smoking_status"]=data["smoking_status"].apply(lambda x:1 if x=="smokes" else(0.66 if x=="smoked" else(0.33 if x=="formerly smoked"else 0)))

l_rate = 0.3
n_epoch = 500
n_hidden = 5
print(data)
scores = evaluate_algorithm(data, back_propagation,  l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


      gender       age  hypertension  heart_disease  ever_married  work_type  \
0          1  0.816895             0              1             1        1.0   
1          0  0.743652             0              0             1        0.5   
2          1  0.975586             0              1             1        1.0   
3          0  0.597168             0              0             1        1.0   
4          0  0.963379             1              0             1        0.5   
...      ...       ...           ...            ...           ...        ...   
5102       0  0.694824             0              0             1        1.0   
5105       0  0.975586             1              0             1        1.0   
5106       0  0.987793             0              0             1        0.5   
5107       0  0.426270             0              0             1        0.5   
5108       1  0.621582             0              0             1        1.0   

      Residence_type  avg_glucose_level

TypeError: can't multiply sequence by non-int of type 'float'